<a href="https://colab.research.google.com/github/irelandclaisen/Urban_Sprawl/blob/master/Measuring_Urban_Sprawl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geemap

     |████████████████████████████████| 399kB 5.1MB/s 
     |████████████████████████████████| 102kB 5.2MB/s 
     |████████████████████████████████| 143kB 8.1MB/s 
     |████████████████████████████████| 2.5MB 8.4MB/s 
     |████████████████████████████████| 102kB 9.0MB/s 
     |████████████████████████████████| 5.1MB 28.5MB/s 
     |████████████████████████████████| 3.3MB 43.1MB/s 
     |████████████████████████████████| 1.1MB 44.2MB/s 
     |████████████████████████████████| 225kB 24.4MB/s 
     |████████████████████████████████| 81kB 8.1MB/s 
     |████████████████████████████████| 552kB 43.6MB/s 
     |████████████████████████████████| 245kB 43.7MB/s 
     |████████████████████████████████| 112kB 41.5MB/s 
     |████████████████████████████████| 122kB 51.3MB/s 
     |████████████████████████████████| 71kB 6.0MB/s 
     |████████████████████████████████| 430kB 20.8MB/s 
     |████████████████████████████████| 71kB 7.6MB/s 
     |████████████████████████████████| 102kB 10.2MB/s 
  C

In [ ]:
import ee
import geemap

In [ ]:
Map = geemap.Map()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=UNyMvLvjB9h-4TOx-bKEzDk4h1HzxVhHpaXdTqHwaac&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g7QspayOCrboJAqWtLmutqp0KFiMMXfR_VqOWQtKuW4bRot0Wn2Pyg

Successfully saved authorization token.


## Seattle

### 2016

In [ ]:
landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
seattle_AOI = ee.Geometry.Rectangle([237, 46.75, 238.5, 48])
landsat_AOI = landsat.filterBounds(seattle_AOI).filterDate('2016-01-01', '2016-12-31').filterMetadata('CLOUD_COVER', 'less_than', 5)
image = landsat_AOI.median().clip(seattle_AOI);

In [ ]:
Map = geemap.Map()
point = ee.Geometry.Point([-122.4439, 47.7538])

vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B2', 'B4', 'B3'],
    'region': seattle_AOI
}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")
Map

In [ ]:
nlcd = ee.Image('USGS/NLCD/NLCD2016').select('landcover').clip(seattle_AOI)
Map.addLayer(nlcd, {}, 'NLCD')
Map

In [ ]:
points = nlcd.sample(**{
    'region': image.geometry(),
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})


In [ ]:
# Use these bands for prediction.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']

# This property of the table stores the land cover labels.
label = 'landcover'

# Overlay the points on the imagery to get training.
training = image.select(bands).sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 30
})

# Train a CART classifier with default parameters.
trained = ee.Classifier.smileCart().train(training, label, bands)

In [ ]:
# Classify the image with the same bands used for training.
result = image.select(bands).classify(trained)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'classfied')
Map

In [ ]:
class_values = nlcd.get('landcover_class_values').getInfo()
class_values

[11,
 12,
 21,
 22,
 23,
 24,
 31,
 41,
 42,
 43,
 51,
 52,
 71,
 72,
 73,
 74,
 81,
 82,
 90,
 95]

In [ ]:
nlcd.getInfo()

In [ ]:
class_palette = nlcd.get('landcover_class_palette').getInfo()
class_palette

['476ba1',
 'd1defa',
 'decaca',
 'd99482',
 'ee0000',
 'ab0000',
 'b3aea3',
 '68ab63',
 '1c6330',
 'b5ca8f',
 'a68c30',
 'ccba7d',
 'e3e3c2',
 'caca78',
 '99c247',
 '78ae94',
 'dcd93d',
 'ab7028',
 'bad9eb',
 '70a3ba']

In [ ]:
class_palette = ['476ba1',
 '68ab63',
 'ab0000',
 'ab0000',
 'ab0000',
 'ab0000',
 '68ab63',
 '68ab63',
 '68ab63',
 '68ab63',
 '68ab63',
 '68ab63',
 '68ab63',
 '68ab63',
 '68ab63',
 '68ab63',
 '68ab63',
 '68ab63',
 '68ab63',
 '68ab63']

In [ ]:
landcover = result.set('classification_class_values', class_values)
landcover = landcover.set('classification_class_palette', class_palette)

In [ ]:
Map.addLayer(landcover, {}, 'Land cover')
Map

In [ ]:
Map.add_legend(labels=['water', 'developed', 'undeveloped'], colors = ['#476ba1', '#ab0000','#68ab63'], )
Map

### 2020

In [ ]:
landsat_AOI = landsat.filterBounds(seattle_AOI).filterDate('2020-01-01', '2020-12-31').filterMetadata('CLOUD_COVER', 'less_than', 5)
image_seattle_2020 = landsat_AOI.median().clip(seattle_AOI);

In [ ]:
Map = geemap.Map()
point = ee.Geometry.Point([-122.4439, 47.7538])

vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B2', 'B4', 'B3'],
    'region': seattle_AOI
}

Map.centerObject(point, 8)
Map.addLayer(image_seattle_2020, vis_params, "Landsat-8")
Map

In [ ]:
# Classify the image with the same bands used for training.
seattle_2020_result = image_seattle_2020.select(bands).classify(trained)
seattle_2020_result_lc = seattle_2020_result.set('classification_class_values', class_values)
seattle_2020_result_lc  = seattle_2020_result_lc.set('classification_class_palette', class_palette)

Map.addLayer(seattle_2020_result_lc, {}, 'classfied')
Map

In [ ]:
urban = seattle_2020_result_lc.eq(24)
areaImage = urban.multiply(ee.Image.pixelArea());


stats = areaImage.reduceRegion(reducer= ee.Reducer.sum(),
  geometry= seattle_AOI,
  scale= 30,
  maxPixels= 1e9
);
print('pixels representing urban: ', stats.get('classification').getInfo(), 'square meters');

pixels representing urban:  287865113.786619 square meters


In [ ]:
print(stats.get('classification').getInfo()/1000000)

259.75570364334163


In [ ]:
print(stats.get('classification').getInfo()/1000000)

In [ ]:
sum_area = 0
for i in [21, 22, 23, 24]:

  urban = seattle_2020_result_lc.eq(i)
  areaImage = urban.multiply(ee.Image.pixelArea());


  stats = areaImage.reduceRegion(reducer= ee.Reducer.sum(),
    geometry= seattle_AOI,
    scale= 30,
    maxPixels= 1e9
  );
  sum_area += stats.get('classification').getInfo()/1000000

print('pixels representing urban: ', sum_area, 'square kilometers in 2020');

pixels representing urban:  3715.8017327559546 square kilometers in 2016


### 2001

In [ ]:
landsat5 = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR")
landsat1_AOI = landsat5.filterBounds(seattle_AOI).filterDate('2001-01-01', '2001-12-31').filterMetadata('CLOUD_COVER', 'less_than', 20)
seattle_2001 = landsat1_AOI.sort('CLOUD_COVER').median().clip(seattle_AOI);

In [ ]:
Map = geemap.Map()
point = ee.Geometry.Point([-122.4439, 47.7538])
# point = ee.Geometry.Point([-87.7719, 41.8799])


vis_params = {
    'min': 0,
    'max': 3000,

    'region': seattle_AOI
}

Map.centerObject(point, 8)
Map.addLayer(seattle_2001, vis_params, "Landsat-8")
Map

In [ ]:
seattle_lc_2001 = ee.Image('USGS/NLCD/NLCD2001').select('landcover').clip(seattle_AOI)
# Classify the image with the same bands used for training.
points_2001 = seattle_lc_2001.sample(**{
    'region': seattle_lc_2001.geometry(),
    'scale': 30,
    'numPixels': 20000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

In [ ]:
# Overlay the points on the imagery to get training.
training = seattle_2001.select(bands).sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 30
})

# Train a CART classifier with default parameters.
#trained_lv = ee.Classifier.smileCart().train(training, label, bands)
trained_seattle_2001 = ee.Classifier.smileRandomForest(10).train(training, label, bands)

In [ ]:
# Classify the image with the same bands used for training.
result_2001 = seattle_2001.select(bands).classify(trained_seattle_2001)
landcover_2001 = result_2001.set('classification_class_values', class_values)
landcover_2001 = landcover_2001.set('classification_class_palette', class_palette)

Map.addLayer(landcover_2001, {}, 'classfied')
Map

In [ ]:
Map.add_legend(labels=['water', 'developed', 'undeveloped'], colors = ['#476ba1', '#ab0000','#68ab63'])
Map

### 1984

In [ ]:
seattle_1984 = landsat5.filterBounds(seattle_AOI).filterDate('1984-01-01', '1984-12-31').filterMetadata('CLOUD_COVER', 'less_than', 18)
image_seattle_1984 = seattle_1984.sort('CLOUD_COVER').median().clip(seattle_AOI);

In [ ]:
Map = geemap.Map()
point = ee.Geometry.Point([-122.4439, 47.7538])


vis_params = {
    'min': 0,
    'max': 3000,

    'region': seattle_AOI
}

Map.centerObject(point, 9)
Map.addLayer(image_seattle_1984, vis_params, "Landsat-8")
Map

In [ ]:
# Classify the image with the same bands used for training.
seattle_1984_result = image_seattle_1984.select(bands).classify(trained_seattle_2001)
seattle_1984_result_lc = seattle_1984_result.set('classification_class_values', class_values)
seattle_1984_result_lc  = seattle_1984_result_lc.set('classification_class_palette', class_palette)

Map.addLayer(seattle_1984_result_lc, {}, 'classfied')
Map

In [ ]:
sum_area_1984 = 0
for i in [21, 22, 23, 24]:

  urban = seattle_1984_result_lc.eq(i)
  areaImage = urban.multiply(ee.Image.pixelArea());


  stats = areaImage.reduceRegion(reducer= ee.Reducer.sum(),
    geometry= seattle_AOI,
    scale= 30,
    maxPixels= 1e9
  );
  sum_area_1984 += stats.get('classification').getInfo()/1000000

print('pixels representing urban: ', sum_area_1984, 'square kilometers in 1984');

pixels representing urban:  1782.1941393901438 square kilometers in 1984


## Las Vegas

### 2016

In [ ]:
landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
las_vegas_AOI = ee.Geometry.Rectangle([-114.6, 35.6, -115.6, 36.5])
las_vegas_2016 = landsat.filterBounds(las_vegas_AOI).filterDate('2016-01-01', '2016-12-31').filterMetadata('CLOUD_COVER', 'less_than', 5)
image = las_vegas_2016.median().clip(las_vegas_AOI);

In [ ]:
Map = geemap.Map()
point = ee.Geometry.Point([-115, 36])
# point = ee.Geometry.Point([-87.7719, 41.8799])


vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B2', 'B4', 'B3'],
    'region': las_vegas_AOI
}

Map.centerObject(point, 9)
Map.addLayer(image, vis_params, "Landsat-8")

In [ ]:
Map

In [ ]:
land_cover_las_vegas = ee.Image('USGS/NLCD/NLCD2016').select('landcover').clip(las_vegas_AOI)
Map.addLayer(land_cover_las_vegas, {}, 'land_cover_las_vegas')
Map.add_legend(builtin_legend='NLCD')
Map

In [ ]:
# Classify the image with the same bands used for training.
points = land_cover_las_vegas.sample(**{
    'region': image.geometry(),
    'scale': 30,
    'numPixels': 10000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

In [ ]:
# Use these bands for prediction.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']


# This property of the table stores the land cover labels.
label = 'landcover'

# Overlay the points on the imagery to get training.
training = image.select(bands).sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 30
})

# Train a CART classifier with default parameters.
#trained_lv = ee.Classifier.smileCart().train(training, label, bands)
trained_lv = ee.Classifier.smileRandomForest(10).train(training, label, bands)

In [ ]:

result = image.select(bands).classify(trained_lv)

landcover_lv2016 = result.set('classification_class_values', class_values)
landcover_lv2016 = landcover_lv2016.set('classification_class_palette', class_palette)
Map.addLayer(landcover_lv2016, {}, 'Land cover')
Map

In [ ]:
sum_area_lv2016 = 0
for i in [21, 22, 23, 24]:

  urban = landcover_lv2016.eq(i)
  areaImage = urban.multiply(ee.Image.pixelArea());


  stats = areaImage.reduceRegion(reducer= ee.Reducer.sum(),
    geometry= las_vegas_AOI,
    scale= 30,
    maxPixels= 1e9
  );
sum_area_lv2016 += stats.get('classification').getInfo()/1000000

print('pixels representing urban: ', sum_area_lv2016, 'square kilometers in 2016');

pixels representing urban:  128.4298669120309 square kilometers in 1984


In [ ]:
# Classify the image with the same bands used for training.
validation = land_cover_las_vegas.sample(**{
    'region': image.geometry(),
    'scale': 30,
    'numPixels': 10000,
    'seed': 1,
    'geometries': True  # Set this to False to ignore geometries
})

In [ ]:
val_set = image.select(bands).sampleRegions(**{
  'collection': validation,
  'properties': [label],
  'scale': 30
})

In [ ]:
validated = val_set.select(bands).classify(trained_lv)

In [ ]:
testAccuracy = validated.errorMatrix('Land_Cover_Type_1', 'classification')
print('Validation error matrix: ', testAccuracy.accuracy().getInfo())

Validation error matrix:  0


### 2020

In [ ]:
las_vegas_2020 = landsat.filterBounds(las_vegas_AOI).filterDate('2020-01-01', '2020-12-31').filterMetadata('CLOUD_COVER', 'less_than', 5)
image_2020 = las_vegas_2020.median().clip(las_vegas_AOI);

In [ ]:
Map = geemap.Map()
point = ee.Geometry.Point([-115, 36])
# point = ee.Geometry.Point([-87.7719, 41.8799])


vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B2', 'B4', 'B3'],
    'region': las_vegas_AOI
}

Map.centerObject(point, 9)
Map.addLayer(image_2020, vis_params, "Landsat-8")
Map

In [ ]:
# Classify the image with the same bands used for training.
lv_2020_result = image_2020.select(bands).classify(trained_lv)
lv_2020_result_lc = lv_2020_result.set('classification_class_values', class_values)
lv_2020_result_lc  = lv_2020_result_lc.set('classification_class_palette', class_palette)

Map.addLayer(lv_2020_result_lc, {}, 'classfied')
Map

In [ ]:
sum_area_lv2020 = 0
for i in [21, 22, 23, 24]:

  urban = lv_2020_result_lc.eq(i)
  areaImage = urban.multiply(ee.Image.pixelArea());


  stats = areaImage.reduceRegion(reducer= ee.Reducer.sum(),
    geometry= las_vegas_AOI,
    scale= 30,
    maxPixels= 1e9
  );
sum_area_lv2020 += stats.get('classification').getInfo()/1000000

print('pixels representing urban: ', sum_area_lv2020, 'square kilometers in 2020');

pixels representing urban:  159.526343509432 square kilometers in 2016


### 2001

In [ ]:
landsat5 = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR")

In [ ]:
lv_2001 = landsat5.filterBounds(las_vegas_AOI).filterDate('2001-01-01', '2001-12-31').filterMetadata('CLOUD_COVER', 'less_than', 5)
image_lv_2001 = lv_2001.sort('CLOUD_COVER').median().clip(las_vegas_AOI);


In [ ]:
Map = geemap.Map()
point = ee.Geometry.Point([-115, 36])


vis_params = {
    'min': 0,
    'max': 3000,

    'region': las_vegas_AOI
}

Map.centerObject(point, 9)
Map.addLayer(image_lv_2001, vis_params, "Landsat-8")
Map

In [ ]:
lc_lv_2001 = ee.Image('USGS/NLCD/NLCD2001').select('landcover').clip(las_vegas_AOI)
Map.addLayer(lc_lv_2001, {}, 'land_cover_las_vegas')
Map.add_legend(builtin_legend='NLCD')
Map

In [ ]:
# Classify the image with the same bands used for training.
points_2001 = lc_lv_2001.sample(**{
    'region': image_lv_2001.geometry(),
    'scale': 30,
    'numPixels': 20000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

In [ ]:
# Overlay the points on the imagery to get training.
training = image_lv_2001.select(bands).sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 30
})

# Train a CART classifier with default parameters.
#trained_lv = ee.Classifier.smileCart().train(training, label, bands)
trained_lv_2001 = ee.Classifier.smileRandomForest(10).train(training, label, bands)

In [ ]:
result = image_lv_2001.select(bands).classify(trained_lv_2001)

landcover_lv2001 = result.set('classification_class_values', class_values)
landcover_lv2001 = landcover_lv2001.set('classification_class_palette', class_palette)
Map.addLayer(landcover_lv2001, {}, 'Land cover')
Map

### 1984

In [ ]:
lv_1984 = landsat5.filterBounds(las_vegas_AOI).filterDate('1984-01-01', '1984-12-31').filterMetadata('CLOUD_COVER', 'less_than', 5)
image_lv_1984 = lv_1984.sort('CLOUD_COVER').median().clip(las_vegas_AOI);

In [ ]:
Map = geemap.Map()
point = ee.Geometry.Point([-115, 36])


vis_params = {
    'min': 0,
    'max': 3000,

    'region': las_vegas_AOI
}

Map.centerObject(point, 9)
Map.addLayer(image_lv_1984, vis_params, "Landsat-8")
Map


In [ ]:
# Classify the image with the same bands used for training.
lv_1984_result = image_lv_1984.select(bands).classify(trained_lv_2001)
lv_1984_result_lc = lv_1984_result.set('classification_class_values', class_values)
lv_1984_result_lc  = lv_1984_result_lc.set('classification_class_palette', class_palette)

Map.addLayer(lv_1984_result_lc, {}, 'classfied')
Map

In [ ]:
sum_area_lv1984 = 0
for i in [21, 22, 23, 24]:

  urban = lv_1984_result_lc.eq(i)
  areaImage = urban.multiply(ee.Image.pixelArea());


  stats = areaImage.reduceRegion(reducer= ee.Reducer.sum(),
    geometry= las_vegas_AOI,
    scale= 30,
    maxPixels= 1e9
  );
sum_area_lv1984 += stats.get('classification').getInfo()/1000000

print('pixels representing urban: ', sum_area_lv1984, 'square kilometers in 2020');

pixels representing urban:  21.7315115757566 square kilometers in 2020


In [ ]:
from IPython.display import Image

In [ ]:
parameters = {'min': 0,
              'max': 1000,
              'dimensions': 200,
              'bands': ['B4', 'B3', 'B2'],
              'region': las_vegas_AOI}

Image(url = lv_1984_result_lc.getThumbUrl(parameters))

In [ ]:
lv_1984_result_lc.get('bands').getInfo()

In [ ]:
lv_1984_result_lc.